In [ ]:
import sys
sys.path.append(r'C:\Users\gustavo\Documents\Data Science\08-GitHub\Portifolio/Classification/titanic')

import pandas as pd
import yaml
import pickle
import os
import json

# Carregando as configurações do arquivo YAML
yaml_path = r"C:\Users\gustavo\Documents\Data Science\08-GitHub\Portifolio\Classification\titanic\src\config.yaml"
with open(yaml_path, "r", encoding="utf-8") as f:
    config = yaml.safe_load(f)


params = {
          'test_data': os.path.join(
              config['output_data']['path'], 
              config['output_data']['file_test']),
          'passanger_id' : config['feat_selection_params']['cols_2_drop'],
          'cols_2_drop' : config['feat_selection_params']['cols_2_drop'],           
          'pipe': config['pipe_feat_eng']['path'],
          'model': config['model']['path'],
          'report': config['save_reports']['path_reports'],
          'submission':config['submission']['path'],
          'target':config['feat_selection_params']['target']
          }
    

    
print('Data submission')
df_test = pd.read_parquet(params['test_data'])

y_test_id = df_test[params['passanger_id']]
df_test.drop(
    columns=params['cols_2_drop'], 
    inplace=True)

pipe_path = os.path.join(
    params['model'],
    f"pipe.pkl")

with open(pipe_path, "rb") as file:
    pipe = pickle.load(file)

df_test_transf = pipe.transform(df_test)    

model_path = os.path.join(
    params['model'],
    f"model.pkl")

with open(model_path, "rb") as file:
    model = pickle.load(file)
    
report_path = os.path.join(
    params['report'],
    "report.json")
# get cols to predict
with open(report_path, "rb") as file:
    report = json.load(file)
    
y_test_id[config['target']] = model.predict(df_test_transf[report['Variaveis mantidas:']])
    


In [ ]:
import pandas as pd
import yaml
import os
import pickle
import matplotlib.pyplot as plt

In [ ]:
from sklearn.inspection import PartialDependenceDisplay

In [ ]:
yaml_path = r"C:\Users\gustavo\Documents\Data Science\08-GitHub\Portifolio\Classification\titanic\src\config.yaml"
with open(yaml_path, "r", encoding="utf-8") as f:
    config = yaml.safe_load(f)

In [ ]:
X_test_feat_sel = os.path.join(
            config['feat_selection']['path'],
            config['feat_selection']['X_test_file_name'])   
    
y_test_feat_sel = os.path.join(
            config['feat_selection']['path'],
            config['feat_selection']['y_test_file_name'])

In [ ]:
params_ = {
        'X_test_feat_sel': X_test_feat_sel,
        'y_test_feat_sel': y_test_feat_sel,
        'model': config['model']['path'],    
        }

In [ ]:
X_test = pd.read_parquet(params_['X_test_feat_sel'])
y_test = pd.read_parquet(params_['y_test_feat_sel']) 

In [ ]:
model_path = os.path.join(
        params_['model'],
        f"model.pkl")
    
with open(model_path, "rb") as file:
        model = pickle.load(file)

In [ ]:
s = pd.Series(
    model.feature_importances_,
    index=model.feature_names_in_
)
s = s.sort_values()
(s
 .plot
 .bar(
     figsize=(20, 8),
     title='Feature Importance'
     )
 )

In [ ]:
list_feat = s[-3:].index

In [ ]:
feature_names_2_explain = []
feature_names_2_explain_cat = []
for feat in list_feat:
    if feat.find('numerical') == 0:
        feature_names_2_explain.append(feat)
    elif feat.find('categorical') == 0:
        feature_names_2_explain_cat.append(feat)

In [ ]:
cols = list(X_test.columns)

In [ ]:
idx_num = []
for col in feature_names_2_explain:
    idx_num.append(cols.index(col))

In [ ]:
idx_cat = []
for col in feature_names_2_explain_cat:
    idx_cat.append(cols.index(col))

In [ ]:
idx = idx_num + idx_cat

In [ ]:
fig, ax = plt.subplots(figsize=(20, 6))
ax.set_title("Parcial dependence plot")

PartialDependenceDisplay.from_estimator(
    estimator=model,
    X=X_test,
    features = idx,
    categorical_features=idx_cat,
    random_state=23,
    ax=ax
)

In [ ]:
fig, ax = plt.subplots(figsize=(20, 6))
ax.set_title("ICE plot")

PartialDependenceDisplay.from_estimator(
    estimator=model,
    X=X_test,
    kind='individual',
    features = idx_num,
    random_state=23,
    ax=ax
);

In [ ]:
fig, ax = plt.subplots(figsize=(20, 6))
ax.set_title("ICE plot")

PartialDependenceDisplay.from_estimator(
    estimator=model,
    X=X_test,
    kind='both',
    features = idx_num,
    random_state=23,
    ax=ax
);

In [ ]:
fig, ax = plt.subplots(figsize=(20, 6))
ax.set_title("ICE plot")

PartialDependenceDisplay.from_estimator(
    estimator=model,
    X=X_test,
    kind='both',
    features = idx_num,
    random_state=23,
    centered=True,
    ax=ax
);